In [11]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, accuracy_score
import tensorflow as tf
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from xgboost import XGBClassifier
from sklearn import svm
import sklearn.metrics as metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Dataset/Twitter_sentiment/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Dataset/Twitter_sentiment/test.csv')

In [4]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Dataset/Twitter_sentiment/submit.csv')

In [5]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [7]:
train.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [9]:
combine=train.append(test,ignore_index=True) #train and test dataset are combined
combine.shape

(49159, 3)


# **Convert to lower case**

In [10]:
combine = combine.lower()

In [13]:
def remove_pattern(input_text,pattern):
    r= re.findall(pattern, input_text)
    for i in r:
        input_text = re.sub(i, '', input_text)
    return input_text
                         

# **Remove Twitter Handles**


In [14]:
combine['tidy_tweet'] = np.vectorize(remove_pattern)(combine['tweet'],"@[\w]*") 
combine.head(10)

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannyâ¦
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here ! i'm it's so #gr8 !


# **Remove Emoji, punctaation, numbers, brackets**

In [15]:
combine['tidy_tweet'] = combine['tidy_tweet'].str.replace("[^a-zA-Z#]"," ")
combine.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0.0,the next school year is the year for exams.ð...,the next school year is the year for exams ...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here i m it s so #gr


# **Remove Short words**

> Indented block



In [16]:
combine['tidy_tweet'] = combine['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) #removing words whose length is less than 3
combine.head(10)

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...,huge fare talking before they leave chaos disp...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0.0,the next school year is the year for exams.ð...,next school year year exams think about that #...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,love land #allin #cavs #champions #cleveland #...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcome here


# **Tokenization**

In [17]:
tokenized_tweet = combine['tidy_tweet'].apply(lambda x:x.split()) #it will split all words by whitespace
tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: tidy_tweet, dtype: object

# **Stemming**

In [18]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) #it will stemmatized all words in tweet

# Combining tokens


In [19]:
#now let's combine these tokens back

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i]) #concat all words into one sentence
combine['tidy_tweet'] = tokenized_tweet

In [20]:
combine.head(10)

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,0.0,bihday your majesty,bihday your majesti
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguid societi #motiv
5,6,0.0,[2/2] huge fan fare and big talking before the...,huge fare talk befor they leav chao disput whe...
6,7,0.0,@user camping tomorrow @user @user @user @use...,camp tomorrow danni
7,8,0.0,the next school year is the year for exams.ð...,next school year year exam think about that #s...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...,love land #allin #cav #champion #cleveland #cl...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...,welcom here


In [ ]:
X_train= combine[0:31962,3]
y = combine[0:31962,1]
X_test = combine[31962:49159,3]

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y))

31962
17197
31962


In [ ]:
vectorizer = CountVectorizer(max_df=0.5)
# vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.5)

X = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=0)

# Linear SVM

In [ ]:
model = LinearSVC(penalty='l2', C=0.5, dual=False, random_state=0, max_iter=1000)
print(model)

LinearSVC(C=0.5, dual=False, random_state=0)


In [ ]:
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print('Accuracy:', accuracy_score(y_pred, y_val))
print("Precision Score: ",metrics.precision_score(y_pred, y_val))
print("Recall Score: ",metrics.recall_score(y_pred, y_val))
print("F1 Score: ", f1_score(y_pred, y_val))

Accuracy: 0.9648052557484749
Precision Score:  0.6151960784313726
Recall Score:  0.786833855799373
F1 Score:  0.6905089408528199


In [ ]:
confusion_matrix(y_pred, y_val)

array([[5917,  157],
       [  68,  251]])

# Random Forest Classifier

In [ ]:
model2 = RandomForestClassifier(n_estimators= 100)
model2.fit(X_train,y_train)
y_pred2 = model2.predict(X_val)
print('Accuracy:', accuracy_score(y_pred2, y_val))
print("Precision Score: ",metrics.precision_score(y_pred2, y_val))
print("Recall Score: ",metrics.recall_score(y_pred2, y_val))
print("F1 Score: ", f1_score(y_pred2, y_val))

Accuracy: 0.9665258876896605
Precision Score:  0.5147058823529411
Recall Score:  0.9292035398230089
F1 Score:  0.6624605678233438


In [ ]:
confusion_matrix(y_pred2, y_val)

array([[5969,  198],
       [  16,  210]])

In [ ]:
model3 = svm.SVC(kernel='poly', degree = 3,probability=True)
model3.fit(X_train,y_train)
y_pred3 = model3.predict(X_val)
print('Accuracy:', accuracy_score(y_pred3, y_val))
print("Precision Score: ",metrics.precision_score(y_pred3, y_val))
print("Recall Score: ",metrics.recall_score(y_pred3, y_val))
print("F1 Score: ", f1_score(y_pred3, y_val))

Accuracy: 0.9491631471922415
Precision Score:  0.23284313725490197
Recall Score:  0.8878504672897196
F1 Score:  0.36893203883495146


In [ ]:
confusion_matrix(y_pred3, y_val)

array([[5973,  313],
       [  12,   95]])

# SVM RBF

In [ ]:
model4 = svm.SVC(kernel='rbf',probability=True)
model4.fit(X_train,y_train)
y_pred4 = model4.predict(X_val)
print('Accuracy:', accuracy_score(y_pred4, y_val))
print("Precision Score: ",metrics.precision_score(y_pred4, y_val))
print("Recall Score: ",metrics.recall_score(y_pred4, y_val))
print("F1 Score: ", f1_score(y_pred4, y_val))

Accuracy: 0.9585484123259815
Precision Score:  0.37254901960784315
Recall Score:  0.9440993788819876
F1 Score:  0.5342706502636204


In [ ]:
confusion_matrix(y_pred4, y_val)

array([[5976,  256],
       [   9,  152]])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
para_grid = {
     'C' : [0.1,0.5,0.6,0.8,1, 10, 100],
     'kernel' :['linear', 'poly', 'sigmoid', 'rbf']
}
m = svm.SVC()
 # Creating a model of Randomized search cv
 # CV = 5 means 5% for testing 
RSCV = RandomizedSearchCV(estimator = m ,  param_distributions= para_grid)
# Training the model 
RSCV.fit(X_train, y_train)

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': [0.1, 0.5, 0.6, 0.8, 1, 10, 100],
                                        'kernel': ['linear', 'poly', 'sigmoid',
                                                   'rbf']})

In [ ]:
RSCV.best_estimator_

SVC(C=0.5, kernel='linear')

# Logistic Regression

In [ ]:
model5 = LogisticRegression(max_iter=1000)
model5.fit(X_train,y_train)
y_pred5 = model5.predict(X_val)
print('Accuracy:', accuracy_score(y_pred5, y_val))
print("Precision Score: ",metrics.precision_score(y_pred5, y_val))
print("Recall Score: ",metrics.recall_score(y_pred5, y_val))
print("F1 Score: ", f1_score(y_pred5, y_val))

Accuracy: 0.9630846238072892
Precision Score:  0.5245098039215687
Recall Score:  0.8359375
F1 Score:  0.6445783132530121


In [ ]:
confusion_matrix(y_pred5, y_val)

array([[5943,  194],
       [  42,  214]])

In [ ]:
model6 = KNN(metric = 'minkowski',n_neighbors=5)
model6.fit(X_train,y_train)
y_pred6 = model6.predict(X_val)
print('Accuracy:', accuracy_score(y_pred6, y_val))
print("Precision Score: ",metrics.precision_score(y_pred6, y_val))
print("Recall Score: ",metrics.recall_score(y_pred6, y_val))
print("F1 Score: ", f1_score(y_pred6, y_val))

Accuracy: 0.9458783043954325
Precision Score:  0.16176470588235295
Recall Score:  0.9428571428571428
F1 Score:  0.27615062761506276


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# train model with full data and predict for new samples
model.fit(X, y)
y_pred_final = model.predict(X_test)

In [ ]:
df = pd.DataFrame()
df['y_pred'] = y_pred_final
df['y_pred'].value_counts()

0    16205
1      992
Name: y_pred, dtype: int64

In [ ]:
# save it to submission csv
submission['label'] = y_pred_final
submission.head()

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Dataset/Twitter_sentiment/submission7.csv', index=False)